In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches


%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [2]:
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [3]:
# This is a callback function to be used with training of Keras models.
# It create an exponential moving average of a model (trainable) weights.
# This functionlity is already available in TensorFlow:
# https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#ExponentialMovingAverage
# and can often be used to get better validation/test performance. For an
# intuitive explantion on why to use this, see 'Model Ensembles" section here:
# http://cs231n.github.io/neural-networks-3/

import numpy as np
import scipy.sparse as sp

from keras import backend as K
from keras.callbacks import Callback
from keras.models import load_model

import sys
import warnings


class ExponentialMovingAverage(Callback):
    """create a copy of trainable weights which gets updated at every
       batch using exponential weight decay. The moving average weights along
       with the other states of original model(except original model trainable
       weights) will be saved at every epoch if save_mv_ave_model is True.
       If both save_mv_ave_model and save_best_only are True, the latest
       best moving average model according to the quantity monitored
       will not be overwritten. Of course, save_best_only can be True
       only if there is a validation set.
       This is equivalent to save_best_only mode of ModelCheckpoint
       callback with similar code. custom_objects is a dictionary
       holding name and Class implementation for custom layers.
       At end of every batch, the update is as follows:
       mv_weight -= (1 - decay) * (mv_weight - weight)
       where weight and mv_weight is the ordinal model weight and the moving
       averaged weight respectively. At the end of the training, the moving
       averaged weights are transferred to the original model.
       """
    def __init__(self, decay=0.999, filepath='temp_weight.hdf5',
                 save_mv_ave_model=True, verbose=0,
                 save_best_only=False, monitor='val_loss', mode='auto',
                 save_weights_only=False, custom_objects={}):
        self.decay = decay
        self.filepath = filepath
        self.verbose = verbose
        self.save_mv_ave_model = save_mv_ave_model
        self.save_weights_only = save_weights_only
        self.save_best_only = save_best_only
        self.monitor = monitor
        self.custom_objects = custom_objects  # dictionary of custom layers
        self.sym_trainable_weights = None  # trainable weights of model
        self.mv_trainable_weights_vals = None  # moving averaged values
        super(ExponentialMovingAverage, self).__init__()

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_train_begin(self, logs={}):
        self.sym_trainable_weights = self.model.trainable_weights
        # Initialize moving averaged weights using original model values
        self.mv_trainable_weights_vals = {x.name: K.get_value(x) for x in
                                          self.sym_trainable_weights}
        if self.verbose:
            print('Created a copy of model weights to initialize moving'
                  ' averaged weights.')

    def on_batch_end(self, batch, logs={}):
        for weight in self.sym_trainable_weights:
            old_val = self.mv_trainable_weights_vals[weight.name]
            self.mv_trainable_weights_vals[weight.name] -= \
                (1.0 - self.decay) * (old_val - K.get_value(weight))

    def on_epoch_end(self, epoch, logs={}):
        """After each epoch, we can optionally save the moving averaged model,
        but the weights will NOT be transferred to the original model. This
        happens only at the end of training. We also need to transfer state of
        original model to model2 as model2 only gets updated trainable weight
        at end of each batch and non-trainable weights are not transferred
        (for example mean and var for batch normalization layers)."""
        if self.save_mv_ave_model:
            filepath = self.filepath.format(epoch=epoch, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best moving averaged model only '
                                  'with %s available, skipping.'
                                  % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('saving moving average model to %s'
                                  % (filepath))
                        self.best = current
                        model2 = self._make_mv_model(filepath)
                        if self.save_weights_only:
                            model2.save_weights(filepath, overwrite=True)
                        else:
                            model2.save(filepath, overwrite=True)
            else:
                if self.verbose > 0:
                    print('Epoch %05d: saving moving average model to %s' % (epoch, filepath))
                model2 = self._make_mv_model(filepath)
                if self.save_weights_only:
                    model2.save_weights(filepath, overwrite=True)
                else:
                    model2.save(filepath, overwrite=True)

    def on_train_end(self, logs={}):
        for weight in self.sym_trainable_weights:
            K.set_value(weight, self.mv_trainable_weights_vals[weight.name])

    def _make_mv_model(self, filepath):
        """ Create a model with moving averaged weights. Other variables are
        the same as original mode. We first save original model to save its
        state. Then copy moving averaged weights over."""
        self.model.save(filepath, overwrite=True)
        model2 = load_model(filepath, custom_objects=self.custom_objects)

        for w2, w in zip(model2.trainable_weights, self.model.trainable_weights):
            K.set_value(w2, self.mv_trainable_weights_vals[w.name])

        return model2


def batch_generator(X, y=None, batch_size=128, shuffle=False):
    index = np.arange(X.shape[0])

    while True:
        if shuffle:
            np.random.shuffle(index)

        batch_start = 0
        while batch_start < X.shape[0]:
            batch_index = index[batch_start:batch_start + batch_size]
            batch_start += batch_size

            X_batch = X[batch_index, :]

            if sp.issparse(X_batch):
                X_batch = X_batch.toarray()

            if y is None:
                yield X_batch
            else:
                yield X_batch, y[batch_index]

In [4]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

# np.log(train['loss'] + 200) provides
# a better score, but let's keep it simple now
train_y = np.array(train['loss'])

print (train_x.shape)
print (train_y.shape)

(188318, 1153)
(188318,)


<img src="http://cdn.rawgit.com/dnkirill/allstate_capstone/master/images/mlp3.svg"></td>

In [9]:
def cross_validate_mlp(mlp_func, nfolds=10):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=1, callbacks=[ExponentialMovingAverage(save_mv_ave_model=False)])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [10]:
from keras import regularizers
from keras.regularizers import l2
def hyper_model():
    model = Sequential()
    model.add(Dense(437, input_dim=train_x.shape[1], init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))
    
    model.add(Dense(182, init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(73, init='he_normal',W_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.233))
    
    model.add(Dense(1, init='he_normal',W_regularizer=l2(0.002)))
    model.compile(loss='mae', optimizer='adadelta')
    return model

In [11]:
cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(437, input_dim=1153, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  """
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(182, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(73, kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_normal", kernel_regularizer

Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 18s 131us/step - loss: 2167.0100 - val_loss: 1212.0037
Epoch 2/30
135588/135588 [==============================] - 17s 127us/step - loss: 1360.8910 - val_loss: 1183.5801
Epoch 3/30
135588/135588 [==============================] - 18s 130us/step - loss: 1333.6907 - val_loss: 1164.5651
Epoch 4/30
135588/135588 [==============================] - 17s 127us/step - loss: 1322.8107 - val_loss: 1156.8181
Epoch 5/30
135588/135588 [==============================] - 17s 128us/step - loss: 1309.4708 - val_loss: 1157.0989
Epoch 6/30
135588/135588 [==============================] - 17s 126us/step - loss: 1301.3325 - val_loss: 1150.9495
Epoch 7/30
135588/135588 [==============================] - 18s 131us/step - loss: 1296.9288 - val_loss: 1152.1262
Epoch 8/30
135588/135588 [==============================] - 18s 131us/step - loss: 1291.9648 - val_loss: 1149.0049
Epoch 9/30
135588/135588 [===

Epoch 10/30
135588/135588 [==============================] - 18s 136us/step - loss: 1271.2397 - val_loss: 1148.9495
Epoch 11/30
135588/135588 [==============================] - 18s 135us/step - loss: 1268.0408 - val_loss: 1146.9558
Epoch 12/30
135588/135588 [==============================] - 19s 136us/step - loss: 1263.0347 - val_loss: 1147.2024
Epoch 13/30
135588/135588 [==============================] - 19s 137us/step - loss: 1255.2314 - val_loss: 1144.7743
Epoch 14/30
135588/135588 [==============================] - 18s 136us/step - loss: 1250.4589 - val_loss: 1144.1285
Epoch 15/30
135588/135588 [==============================] - 18s 135us/step - loss: 1249.6973 - val_loss: 1143.0931
Epoch 16/30
135588/135588 [==============================] - 18s 135us/step - loss: 1240.0072 - val_loss: 1142.0111
Epoch 17/30
135588/135588 [==============================] - 18s 134us/step - loss: 1235.7677 - val_loss: 1141.8242
Epoch 18/30
135588/135588 [==============================] - 18s 135us/s

135588/135588 [==============================] - 19s 141us/step - loss: 1217.1953 - val_loss: 1137.9409
Epoch 20/30
135588/135588 [==============================] - 19s 141us/step - loss: 1213.8227 - val_loss: 1138.5104
Epoch 21/30
135588/135588 [==============================] - 20s 145us/step - loss: 1209.3225 - val_loss: 1138.5747
Epoch 22/30
135588/135588 [==============================] - 20s 145us/step - loss: 1203.1665 - val_loss: 1141.1783
Epoch 23/30
135588/135588 [==============================] - 19s 142us/step - loss: 1206.2553 - val_loss: 1139.0098
Epoch 24/30
135588/135588 [==============================] - 19s 143us/step - loss: 1201.3716 - val_loss: 1138.2706
Epoch 25/30
135588/135588 [==============================] - 19s 143us/step - loss: 1198.7591 - val_loss: 1139.2440
Epoch 26/30
135588/135588 [==============================] - 19s 141us/step - loss: 1191.3677 - val_loss: 1139.1006
Epoch 27/30
135588/135588 [==============================] - 19s 142us/step - loss: 

135588/135588 [==============================] - 19s 142us/step - loss: 1190.5755 - val_loss: 1139.0932
Epoch 29/30
135588/135588 [==============================] - 19s 143us/step - loss: 1186.1293 - val_loss: 1138.0854
Epoch 30/30
135588/135588 [==============================] - 19s 142us/step - loss: 1182.4782 - val_loss: 1138.7171
Fold 6, MAE: 1138.8995213783282
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 20s 148us/step - loss: 2195.4001 - val_loss: 1222.9675
Epoch 2/30
135588/135588 [==============================] - 20s 146us/step - loss: 1348.9641 - val_loss: 1180.9750
Epoch 3/30
135588/135588 [==============================] - 20s 146us/step - loss: 1330.3625 - val_loss: 1172.3513
Epoch 4/30
135588/135588 [==============================] - 20s 146us/step - loss: 1316.8404 - val_loss: 1157.2486
Epoch 5/30
135588/135588 [==============================] - 20s 147us/step - loss: 1309.1679 - val_loss: 1157.2928
Epoch 

Epoch 7/30
135589/135589 [==============================] - 20s 151us/step - loss: 1351.7093 - val_loss: 1160.1289
Epoch 8/30
135589/135589 [==============================] - 20s 147us/step - loss: 1338.7285 - val_loss: 1155.9709
Epoch 9/30
135589/135589 [==============================] - 20s 150us/step - loss: 1330.4124 - val_loss: 1156.8836
Epoch 10/30
135589/135589 [==============================] - 20s 150us/step - loss: 1320.3980 - val_loss: 1156.9649
Epoch 11/30
135589/135589 [==============================] - 21s 152us/step - loss: 1310.6417 - val_loss: 1151.5215
Epoch 12/30
135589/135589 [==============================] - 20s 148us/step - loss: 1296.6134 - val_loss: 1150.1943
Epoch 13/30
135589/135589 [==============================] - 20s 150us/step - loss: 1289.2152 - val_loss: 1151.0073
Epoch 14/30
135589/135589 [==============================] - 20s 150us/step - loss: 1282.8837 - val_loss: 1152.2900
Epoch 15/30
135589/135589 [==============================] - 21s 153us/step

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.

In [ ]:
'l3-nn': {
        'predictions': l2_predictions,
        'n_bags': 4,
        'model': Keras(nn_lr, lambda: {'l1': 1e-5, 'l2': 1e-5, 'n_epoch': 30, 'batch_size': 128, 'optimizer': SGD(3e-2, momentum=0.8, nesterov=True, decay=3e-5), 'callbacks': [ExponentialMovingAverage(save_mv_ave_model=False)]}),
    },

In [ ]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [ ]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adam','adadelta']),
         'wdecay':hp.choice('wdecay', np.linspace(0.00001,0.1,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], init='he_normal',
                        W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], init='he_normal',W_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, init='he_normal',W_regularizer=l2(params['wdecay'])))
        model.compile(loss='mae', optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.4210526315789474, 'hidden1_units': 455, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 272, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 69, 'optimizer': 'adadelta', 'wdecay': 0.032330000000000005}
Fold 0, MAE: 1163.148839039124
Fold 1, MAE: 1159.9899238609737
Fold 2, MAE: 1140.9771315408389
3-fold CV score: 1154.7052981469787
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.43333333333333335, 'hidden2_units': 134, 'hidden3_dropout': 0.10000000000000001, 'hidden3_units': 30, 'optimizer': 'adadelta', 'wdecay': 0.082830000000000001}
Fold 0, MAE: 1181.2090776051534
Fold 1, MAE: 1175.7242010975745
Fold 2, MAE: 1149.1617817128606
3-fold CV score: 1168.698353471863
Model Testing: {'hidden1_dropout': 0.57894736842105265, 'hidden1_units': 541, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 113, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 22, 'optimizer': 'adam', '

Fold 0, MAE: 1160.1148852933434
Fold 1, MAE: 1153.5388360438872
Fold 2, MAE: 1146.284142179796
3-fold CV score: 1153.312621172342
Model Testing: {'hidden1_dropout': 0.44210526315789478, 'hidden1_units': 325, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 258, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 59, 'optimizer': 'adadelta', 'wdecay': 0.058590000000000003}
Fold 0, MAE: 1159.533359263806
Fold 1, MAE: 1154.5401700530679
Fold 2, MAE: 1152.34695090863
3-fold CV score: 1155.473493408501
Model Testing: {'hidden1_dropout': 0.43157894736842106, 'hidden1_units': 377, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 189, 'hidden3_dropout': 0.27777777777777779, 'hidden3_units': 63, 'optimizer': 'adadelta', 'wdecay': 0.081820000000000004}
Fold 0, MAE: 1162.9307342647628
Fold 1, MAE: 1177.8290937843656
Fold 2, MAE: 1148.0584482485735
3-fold CV score: 1162.9394254325673
Model Testing: {'hidden1_dropout': 0.55789473684210522, 'hidden1_units': 455, 'hidden2_dropout

Fold 0, MAE: 1163.881212321844
Fold 1, MAE: 1164.1301011722203
Fold 2, MAE: 1150.2623383121809
3-fold CV score: 1159.4245506020818
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 532, 'hidden2_dropout': 0.40000000000000002, 'hidden2_units': 251, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adadelta', 'wdecay': 0.074749999999999997}
Fold 0, MAE: 1161.926003949864
Fold 1, MAE: 1171.028653969578
Fold 2, MAE: 1147.243582406554
3-fold CV score: 1160.0660801086653
Model Testing: {'hidden1_dropout': 0.5473684210526315, 'hidden1_units': 300, 'hidden2_dropout': 0.33333333333333337, 'hidden2_units': 127, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 55, 'optimizer': 'adam', 'wdecay': 0.0020300000000000001}
Fold 0, MAE: 1157.4905136832626
Fold 1, MAE: 1157.7959576055189
Fold 2, MAE: 1145.165175664648
3-fold CV score: 1153.4838823178097
Model Testing: {'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.